In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-player-contact-detection/test_baseline_helmets.csv
/kaggle/input/nfl-player-contact-detection/sample_submission.csv
/kaggle/input/nfl-player-contact-detection/train_baseline_helmets.csv
/kaggle/input/nfl-player-contact-detection/test_player_tracking.csv
/kaggle/input/nfl-player-contact-detection/train_player_tracking.csv
/kaggle/input/nfl-player-contact-detection/train_video_metadata.csv
/kaggle/input/nfl-player-contact-detection/test_video_metadata.csv
/kaggle/input/nfl-player-contact-detection/train_labels.csv
/kaggle/input/nfl-player-contact-detection/test/58172_003247_All29.mp4
/kaggle/input/nfl-player-contact-detection/test/58172_003247_Sideline.mp4
/kaggle/input/nfl-player-contact-detection/test/58172_003247_Endzone.mp4
/kaggle/input/nfl-player-contact-detection/test/58168_003392_Endzone.mp4
/kaggle/input/nfl-player-contact-detection/test/58168_003392_Sideline.mp4
/kaggle/input/nfl-player-contact-detection/test/58168_003392_All29.mp4
/kaggle/input/nfl-player-con

This notebook contains training and prediction of LGBM utilising Bayesian Hyperparameter optimisation (if thats possible if not this is going to be with CNN and Bayesian Hyper Opt

# Is the submission file the test labels???

# Importing Dependencies 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gc

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, PowerTransformer
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold
from sklearn.metrics import log_loss, auc, accuracy_score, roc_auc_score, matthews_corrcoef

# Importing Data

Function to reduce the memory requirement of the data. Credit Pavan Kumar D https://www.kaggle.com/questions-and-answers/148011.

In [3]:
def reduce_mem_usage(train_data):
    """ iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    start_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in train_data.columns:
        col_type = train_data[col].dtype

        if col_type != object:
            c_min = train_data[col].min()
            c_max = train_data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train_data[col] = train_data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train_data[col] = train_data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train_data[col] = train_data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train_data[col] = train_data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    train_data[col] = train_data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train_data[col] = train_data[col].astype(np.float32)
                else:
                    train_data[col] = train_data[col].astype(np.float64)
        else:
            train_data[col] = train_data[col].astype('category')

    end_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return train_data

**Train Player Tracking**

In [4]:
train_player_tracking = pd.read_csv("/kaggle/input/nfl-player-contact-detection/train_player_tracking.csv")
train_player_tracking = reduce_mem_usage(train_player_tracking)
display(train_player_tracking.head())
display(train_player_tracking.shape)

Memory usage of dataframe is 175.49 MB
Memory usage after optimization is: 50.24 MB
Decreased by 71.4%


,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
0,58580_001136,58580,1136,44830,2021-10-10T21:08:20.900Z,-108,away,CB,22,61.59375,42.593750,1.110352,0.109985,320.250000,264.000,0.709961,-0.640137
1,58580_001136,58580,1136,47800,2021-10-10T21:08:20.900Z,-108,away,DE,97,59.46875,26.812500,0.229980,0.010002,346.750000,247.125,1.290039,0.899902
2,58580_001136,58580,1136,52444,2021-10-10T21:08:20.900Z,-108,away,FS,29,72.18750,31.453125,0.609863,0.059998,11.773438,247.750,0.629883,-0.330078
3,58580_001136,58580,1136,46206,2021-10-10T21:08:20.900Z,-108,home,TE,86,57.37500,22.125000,0.370117,0.040009,127.875000,63.625,0.689941,0.620117
4,58580_001136,58580,1136,52663,2021-10-10T21:08:20.900Z,-108,away,ILB,48,63.25000,27.500000,0.509766,0.049988,183.625000,253.750,0.310059,0.310059


(1353053, 17)

**Train Labels**

In [5]:
train_labels = pd.read_csv("/kaggle/input/nfl-player-contact-detection/train_labels.csv")
train_labels = reduce_mem_usage(train_labels)
display(train_labels.head())
display(train_labels.shape)

Memory usage of dataframe is 252.16 MB
Memory usage after optimization is: 242.30 MB
Decreased by 3.9%


,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392,2020-09-11T03:01:48.100Z,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392,2020-09-11T03:01:48.100Z,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,47944,0


(4721618, 7)

**Test Player Tracking**

In [6]:
test_player_tracking = pd.read_csv("/kaggle/input/nfl-player-contact-detection/test_player_tracking.csv")
test_player_tracking = reduce_mem_usage(test_player_tracking)
display(test_player_tracking.head())
display(test_player_tracking.shape)

Memory usage of dataframe is 1.93 MB
Memory usage after optimization is: 0.50 MB
Decreased by 73.9%


,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
0,58172_003247,58172,3247,41937,2020-09-13T19:30:20.200Z,-272,home,MLB,57,64.25000,11.289062,4.820312,0.459961,20.734375,12.429688,3.130859,3.000000
1,58172_003247,58172,3247,45345,2020-09-13T19:30:20.200Z,-272,away,ILB,50,76.00000,36.750000,3.150391,0.320068,179.500000,184.875000,1.870117,-1.809570
2,58172_003247,58172,3247,46205,2020-09-13T19:30:20.200Z,-272,home,DE,98,65.62500,15.742188,1.799805,0.199951,6.730469,339.750000,0.689941,-0.680176
3,58172_003247,58172,3247,43406,2020-09-13T19:30:20.200Z,-272,home,WR,14,36.34375,14.937500,1.809570,0.180054,330.500000,344.000000,0.620117,0.389893
4,58172_003247,58172,3247,48233,2020-09-13T19:30:20.200Z,-272,away,RB,45,76.12500,32.156250,3.359375,0.340088,166.625000,180.500000,0.879883,-0.870117


(14872, 17)

Might Want to display some of the testing videos to show what we're predicting

# Merging Data

Converting Labels to strings so merges can be done correctly

In [7]:
train_labels[["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"]]=train_labels[["game_play", "step", "nfl_player_id_1", "nfl_player_id_2"]].astype("str")
train_player_tracking[["game_play", "step", "nfl_player_id"]]=train_player_tracking[["game_play", "step", "nfl_player_id"]].astype("str")

We first create 2 temporary train datasets that combine the player tracking data and the train labels. There are 2 dataframes as one is joined on nfl_player_id_1 and the other on nfl_player_id_2.   
This is done so that each record is a complete account of each contact, with the data for player 1 and player 2 combined in the same row.

In [8]:
temp_train_1 = pd.merge(train_labels, train_player_tracking, left_on = ["game_play","step","nfl_player_id_1"], right_on =["game_play","step","nfl_player_id"], how="left")
temp_train_2 = pd.merge(train_labels, train_player_tracking, left_on = ["game_play","step","nfl_player_id_2"], right_on=["game_play","step","nfl_player_id"],how="left")
print(f"Temp Train 1 shape: {temp_train_1.shape}")
print(f"Temp Train 2 shape: {temp_train_2.shape}")

Temp Train 1 shape: (4721618, 22)
Temp Train 2 shape: (4721618, 22)


In [9]:
temp_train_2.head() #Checking the merge has worked correctly

,contact_id,game_play,datetime_x,step,nfl_player_id_1,nfl_player_id_2,contact,game_key,play_id,nfl_player_id,...,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
0,58168_003392_0_38590_43854,58168_003392,2020-09-11T03:01:48.100Z,0,38590,43854,0,58168.0,3392.0,43854,...,OLB,57.0,42.00000,16.796875,0.740234,0.059998,264.000,294.75,1.740234,1.740234
1,58168_003392_0_38590_41257,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41257,0,58168.0,3392.0,41257,...,CB,21.0,45.78125,15.593750,0.669922,0.070007,180.375,271.00,0.340088,0.080017
2,58168_003392_0_38590_41944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41944,0,58168.0,3392.0,41944,...,DT,92.0,42.00000,22.843750,0.680176,0.080017,234.125,282.00,0.810059,0.810059
3,58168_003392_0_38590_42386,58168_003392,2020-09-11T03:01:48.100Z,0,38590,42386,0,58168.0,3392.0,42386,...,ILB,55.0,45.87500,23.890625,0.609863,0.059998,221.250,269.50,1.110352,1.099609
4,58168_003392_0_38590_47944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,47944,0,58168.0,3392.0,47944,...,DE,94.0,42.09375,26.578125,0.199951,0.029999,240.125,266.00,1.629883,1.230469


In [10]:
temp_train_1.columns[7:] #Printing all columns unique to the player tracking data 

Index(['game_key', 'play_id', 'nfl_player_id', 'datetime_y', 'team',
       'position', 'jersey_number', 'x_position', 'y_position', 'speed',
       'distance', 'direction', 'orientation', 'acceleration', 'sa'],
      dtype='object')

**We then rename the columns in each temporary dataset with a suffix of \_1 and \_2 so that the columns for each player ID are distinct.**

In [11]:
temp_train_1 = temp_train_1.rename(columns={c: c+"_1" for c in temp_train_1.columns[7:]})
temp_train_2 = temp_train_2.rename(columns={c: c+"_2" for c in temp_train_2.columns[7:]})

In [12]:
merge_train = pd.merge(temp_train_1,temp_train_2, on = ["contact_id","step"], how="left")
merge_train

,contact_id,game_play_x,datetime_x_x,step,nfl_player_id_1_x,nfl_player_id_2_x,contact_x,game_key_1,play_id_1,nfl_player_id_1_x,...,position_2,jersey_number_2,x_position_2,y_position_2,speed_2,distance_2,direction_2,orientation_2,acceleration_2,sa_2
0,58168_003392_0_38590_43854,58168_003392,2020-09-11T03:01:48.100Z,0,38590,43854,0,58168,3392,38590,...,OLB,57.0,42.00000,16.796875,0.740234,0.059998,264.000,294.75,1.740234,1.740234
1,58168_003392_0_38590_41257,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41257,0,58168,3392,38590,...,CB,21.0,45.78125,15.593750,0.669922,0.070007,180.375,271.00,0.340088,0.080017
2,58168_003392_0_38590_41944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41944,0,58168,3392,38590,...,DT,92.0,42.00000,22.843750,0.680176,0.080017,234.125,282.00,0.810059,0.810059
3,58168_003392_0_38590_42386,58168_003392,2020-09-11T03:01:48.100Z,0,38590,42386,0,58168,3392,38590,...,ILB,55.0,45.87500,23.890625,0.609863,0.059998,221.250,269.50,1.110352,1.099609
4,58168_003392_0_38590_47944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,47944,0,58168,3392,38590,...,DE,94.0,42.09375,26.578125,0.199951,0.029999,240.125,266.00,1.629883,1.230469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4721613,58582_003121_91_48220_G,58582_003121,2021-10-12T02:42:29.100Z,91,48220,G,0,58582,3121,48220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4721614,58582_003121_91_47906_G,58582_003121,2021-10-12T02:42:29.100Z,91,47906,G,0,58582,3121,47906,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4721615,58582_003121_91_38557_G,58582_003121,2021-10-12T02:42:29.100Z,91,38557,G,0,58582,3121,38557,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4721616,58582_003121_91_47872_G,58582_003121,2021-10-12T02:42:29.100Z,91,47872,G,0,58582,3121,47872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
merge_train.isna().sum()

contact_id                0
game_play_x               0
datetime_x_x              0
step                      0
nfl_player_id_1_x         0
nfl_player_id_2_x         0
contact_x                 0
game_key_1                0
play_id_1                 0
nfl_player_id_1_x         0
datetime_y_1              0
team_1                    0
position_1                0
jersey_number_1           0
x_position_1              0
y_position_1              0
speed_1                   0
distance_1                0
direction_1               0
orientation_1             0
acceleration_1            0
sa_1                      0
game_play_y               0
datetime_x_y              0
nfl_player_id_1_y         0
nfl_player_id_2_y         0
contact_y                 0
game_key_2           410633
play_id_2            410633
nfl_player_id_2_y    410633
datetime_y_2         410633
team_2               410633
position_2           410633
jersey_number_2      410633
x_position_2         410633
y_position_2        

In [14]:
del temp_train_1,temp_train_2 #freeing up RAM
gc.collect()

87

Also be sure to split data into training and val data (k-fold) prob 80-20

# Modelling

### Bayesian Hyperparameter Optimisation

# Prediction and Evaluation